# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"linked_variables":{}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,95.322121,0.564880,1234,95.887002,94.757241,191.774004,189.514482
6,758.467395,-0.474059,1234,757.993336,758.941454,1515.986673,1517.882908
7,166.168170,-0.710272,1234,165.457898,166.878443,330.915796,333.756885
8,70.489840,-0.226139,1234,70.263702,70.715979,140.527404,141.431958
9,133.900825,0.692359,1234,134.593184,133.208466,269.186368,266.416932
10,204.799021,-0.308871,1234,204.490150,205.107892,408.980300,410.215785
11,430.020288,0.482720,1234,430.503008,429.537568,861.006017,859.075136
12,36.606955,-0.673826,1234,35.933129,37.280781,71.866259,74.561562
13,312.510054,-0.708095,1234,311.801959,313.218150,623.603918,626.436299
14,517.145960,0.181975,1234,517.327935,516.963985,1034.655869,1033.927971


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-95.887002,94.757241
6,-757.993336,758.941454
7,-165.457898,166.878443
8,-70.263702,70.715979
9,-134.593184,133.208466
10,-204.490150,205.107892
11,-430.503008,429.537568
12,-35.933129,37.280781
13,-311.801959,313.218150
14,-517.327935,516.963985


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-95.887002,94.757241
6,-757.993336,758.941454
7,-165.457898,166.878443
8,-70.263702,70.715979
9,-134.593184,133.208466
10,-204.490150,205.107892
11,-430.503008,429.537568
12,-35.933129,37.280781
13,-311.801959,313.218150
14,-517.327935,516.963985


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -95.88700185,   94.75724087],
       [-757.99333636,  758.94145394],
       [-165.45789792,  166.8784426 ],
       [ -70.26370192,   70.71597896],
       [-134.59318416,  133.20846586],
       [-204.49015004,  205.1078924 ],
       [-430.50300842,  429.53756817],
       [ -35.93312926,   37.28078107],
       [-311.80195912,  313.21814972],
       [-517.32793458,  516.96398542]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,189.774004,-189.514482
6,1513.986673,-1517.882908
7,328.915796,-333.756885
8,138.527404,-141.431958
9,267.186368,-266.416932
10,406.980300,-410.215785
11,859.006017,-859.075136
12,69.866259,-74.561562
13,621.603918,-626.436299
14,1032.655869,-1033.927971


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
